## Scenario: Token Issuers can maintain multiple Token Status Lists

In [1]:
from helpers import fixtures, utils
from helpers.utils import tc
import requests
import uuid
import random

### Configure the base URL of the Status List Server

In [2]:
BASE_URL = utils.get_base_url()
print(f"Using base URL: {BASE_URL}")

Using base URL: http://localhost:8000


### Test the Health Check Endpoint (GET /health)

In [3]:
health_endpoint = f"{BASE_URL}/health"
print(f"Testing GET: {health_endpoint}")

response = requests.get(health_endpoint)
tc.assertEqual(response.status_code, 200, "Health check failed")
tc.assertEqual("OK", response.json().get("status"))

print("Health check successful! Server is running.")

Testing GET: http://localhost:8000/health
Health check successful! Server is running.


### Two Issuers register on the Status List Server by publish credentials

In [4]:
gondwana_digital_pole = fixtures.get_gondwana_digital_pole_issuer()
scott_holdings = fixtures.get_scott_holdings_issuer()

for issuer_data in [gondwana_digital_pole, scott_holdings]:
    print("Proceeding with Issuer:", issuer_data.get('label'))
    
    credentials_endpoint = f"{BASE_URL}/credentials"
    print(f"Testing POST: {credentials_endpoint}")
    
    payload = {
      "issuer": issuer_data.get('label'),
      "public_key": issuer_data.get('keypair').get('public_key'),
      "alg": issuer_data.get('keypair').get('alg')
    }
    
    response = requests.post(credentials_endpoint, json=payload)
    tc.assertEqual(response.status_code, 202, "Failed to publish credentials")
    print("Credentials published successfully!\n")

Proceeding with Issuer: gondwana-digital-pole-gyks1
Testing POST: http://localhost:8000/credentials
Credentials published successfully!

Proceeding with Issuer: scott-holdings-ahmiu
Testing POST: http://localhost:8000/credentials
Credentials published successfully!



### They publish multiple status lists

In [5]:
# Since status lists are so far not linked to issuers, just assume they are.
# Below, each issuer publishes three lists.

status_publish_endpoint = f"{BASE_URL}/statuslists/publish"
print(f"Testing POST: {status_publish_endpoint}")

for issuer_data in [gondwana_digital_pole, scott_holdings]:
    for _ in range(3): # each issuer publishes three lists
        status_list_id = str(uuid.uuid4())
        print(f"{issuer_data.get('label')} publishes status list {status_list_id}")
        
        response = requests.post(status_publish_endpoint, json={
            "list_id": status_list_id,
            "bits": random.choice([1, 2, 4, 8]),
            "updates": [{"index": i, "status": "VALID"} for i in range(random.randint(1, 5))]
        })
        
        tc.assertEqual(response.status_code, 201, "Failed to publish token statuses")

Testing POST: http://localhost:8000/statuslists/publish
gondwana-digital-pole-gyks1 publishes status list 3dc3286a-93f1-4aed-9436-45e697cd695a
gondwana-digital-pole-gyks1 publishes status list 35806520-6081-4119-924f-92551631b509
gondwana-digital-pole-gyks1 publishes status list fc43156c-7482-46f7-9b8c-d76e3638d48f
scott-holdings-ahmiu publishes status list 17e6bf56-6dd9-4d7e-a7e8-10088c861660
scott-holdings-ahmiu publishes status list 655b5496-185d-4137-b8cd-fb11e274168b
scott-holdings-ahmiu publishes status list 76803181-65ad-4742-ad07-95a4541fd64f
